In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

## Load the BERT Classifier and Tokenizer along with Input modules

In [2]:
# model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading: 100%|██████████| 226k/226k [00:01<00:00, 172kB/s]  
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 8.10kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 286kB/s]


## Load CSV, test and validation datasets

In [3]:
test = pd.read_csv('./test.csv')
validation = pd.read_csv('./validation.csv')
test = test.iloc[: , 1:]
validation = validation.iloc[: , 1:]

In [4]:
path = './inputCsv.csv'
train = pd.read_csv(path)

In [5]:
train['sentiment'] = train['sentiment'].replace('positive', 0)
train['sentiment'] = train['sentiment'].replace('negative', 1)
train['sentiment'] = train['sentiment'].replace('neutral', 2)
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train

,DATA_COLUMN,LABEL_COLUMN
0,Today was the best day ever!,0
1,"Do not talk to me, I am in a bad mood",1
2,I do not live in Nicosia,2
3,I am very optimistic for this effort,0
4,The match was exciting,0
5,The weather is mild,2
6,I hate my boss,1
7,I hope my teams wins tonight,0
8,It’s great to be back 😃 #RolandGarros,0
9,#NewProfilePic,2


## Create input sequences

In [6]:
def convert_data_to_examples_single(inputDataset, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = inputDataset.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)  
  return train_InputExamples


def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

    

In [7]:
DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [8]:
test_inputExamples = convert_data_to_examples_single(test, DATA_COLUMN, LABEL_COLUMN)
test_data = convert_examples_to_tf_dataset(list(test_inputExamples), tokenizer)
test_data = test_data.batch(32)

validation_InputExamples = convert_data_to_examples_single(validation, DATA_COLUMN, LABEL_COLUMN)
validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## Retrain BERT model

In [9]:
model_save_name = 'BERTModel'
path = F"./{model_save_name}" 

loaded_model = TFBertForSequenceClassification.from_pretrained(path, local_files_only=True)

loaded_model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./BERTModel.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [10]:
train_InputExamples = convert_data_to_examples_single(train, DATA_COLUMN, LABEL_COLUMN)
train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.batch(32).repeat(2)

In [11]:
loaded_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [12]:
# loaded_model.evaluate(test_data)

In [13]:
loaded_model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
2/2 [==============================] - 563s 527s/step - loss: 0.0344 - accuracy: 1.0000 - val_loss: 0.9095 - val_accuracy: 0.7665
Epoch 2/2
2/2 [==============================] - 526s 518s/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.9766 - val_accuracy: 0.7526


In [14]:
loaded_model.evaluate(test_data)

86/86 [==============================] - 527s 6s/step - loss: 1.0514 - accuracy: 0.7344


[1.0513725280761719, 0.7343522310256958]

## Save new model

In [15]:
model_save_name = 'BERTModel'
path = F"./{model_save_name}" 
loaded_model.save_pretrained(path)

## Make Predictions with the new Model

In [16]:
pred_sentences = ['Today was the best day ever!',
                  'Do not talk to me, I am in a bad mood',
                  'I do not live in Nicosia',
                  'I am very optimistic for this effort']

In [17]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = loaded_model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Positive','Negative','Neutral']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

Today was the best day ever! : 
 Positive
Do not talk to me, I am in a bad mood : 
 Negative
I do not live in Nicosia : 
 Neutral
I am very optimistic for this effort : 
 Positive
